<a href="https://colab.research.google.com/github/RifatMuhtasim/Apache_Spark/blob/main/1.06.PySpark_Mlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gdown
import os
import zipfile

# Replace 'output_path' with the path where you want to save the file
output_path = 'dataset.zip'

if os.path.exists(output_path):
    print("File exists!")

else:
    print("File does not exist.")
    # Replace 'file_id' with the ID of your file in Google Drive
    file_id = '1matQHY1Q4xHT4m_jxVSZ8Oefgk6ZiDi2'
    gdown.download(f'https://drive.google.com/uc?id={file_id}', output_path, quiet=False)

    # Path to your .zip file (Must Change. Same as the Output Path)
    zip_file_path = '/content/dataset.zip'

    # Extract the contents of the .zip file to the root directory
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall('/content/')

    # List the contents of the root directory
    extracted_files = !ls -a /content/
    print("Files extracted to root directory:", extracted_files)

File exists!


In [2]:
#Load Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MyApp").getOrCreate()
spark

In [3]:
#Load dataset
ds = spark.read.csv("test1.csv", header= True, inferSchema= True)
ds.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
#Show all columns name
ds.columns

['Name', 'age', 'Experience', 'Salary']

# Machine Learning Using PySpark

In [5]:
from pyspark.ml.feature import VectorAssembler
featureassebmbler = VectorAssembler(inputCols=['age', 'Experience'], outputCol="Independent Features")

In [6]:
#Crate a new column with vector
output = featureassebmbler.transform(ds)
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [7]:
finalized_data = output.select("Independent Features", "Salary")
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [8]:
#Split the dataset
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
train_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [21.0,1.0]| 15000|
|          [29.0,4.0]| 20000|
|          [30.0,8.0]| 25000|
|         [31.0,10.0]| 30000|
+--------------------+------+



## Linear Regression Model

In [9]:
from pyspark.ml.regression import LinearRegression

regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

In [10]:
#Prediction
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|16538.461538461514|
|          [24.0,3.0]| 20000|18128.205128205103|
+--------------------+------+------------------+

